# <p style="text-align: center;">EE 379K: Data Science Lab</p>
# <p style="text-align: center;">Lab 3 - 9/25/17</p>
## <p style="text-align: center;">Rachel Chen and Kevin Yee</p>
### <p style="text-align: center;"> rjc2737 and kjy252</p>


![](http://i.markdownnotes.com/image_KBazLVB.png)

![](http://i.markdownnotes.com/image_yVAcZ2O.png)

In [1]:
from cStringIO import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

#converts pdf, returns its text content as a string
def convert(faname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)
    print faname
    infile = open(faname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

In [1]:
import urllib
from urllib import urlopen
import re
from bs4 import BeautifulSoup

# path = "C:\Users\Kevin\Google Drive\Ipython\Data Science Lab\Lab3\scrapedPDF\\"
url = "http://proceedings.mlr.press/v70/"
content = urllib.urlopen(url).read()

#sanity check
print content

<!DOCTYPE html>
<html>

  <head>
  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  
  <title>Proceedings of Machine Learning Research</title>
  <meta name="description" content="Proceedings of the 34th International Conference on Machine Learning
  Held in International Convention Centre, Sydney, Australia on 06-11 August 2017

Published as Volume 70 by the Proceedings of Machine Learning Research on 17 July 2017.

Volume Edited by:
  Doina Precup
  Yee Whye Teh

Series Editors:
  Neil D. Lawrence
  Mark Reid
">
  <link rel="stylesheet" type="text/css" href="../css/main.css" />
  
  
<script>
  (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
  (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
  m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
  })(window,document,'script','//www.google-analytics.

In [40]:
soup=BeautifulSoup(content,"html.parser")
html = soup.find_all(href=re.compile("http://.*17[a-z].pdf"))

#more sanity check 
print html[0]['href']

http://proceedings.mlr.press/v70/achab17a/achab17a.pdf


In [44]:
for i in range(374,len(html)):
    pdf_url = html[i]['href']
    f = open(path + str(i)+".pdf", 'wb')
    pdf = urlopen(pdf_url)
    f.write(pdf.read())
    f.close()

In [3]:
#converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir
def convertMultiple(pdfDir, txtDir):
    if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
    for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
        fileExtension = pdf.split(".")[-1]
        
        pdfFilename = pdfDir + pdf 
        text = convert(pdfFilename) #get string of text content of pdf
        textFilename = txtDir + pdf + ".txt"
        textFile = open(textFilename, "w") #make text file
        textFile.write(text) #write text to text file

pdfDire = "C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/"
txtDir = "C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedTXT/"
convertMultiple(pdfDire, txtDir)

C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/0.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/1.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/10.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/100.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/101.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/102.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/103.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/104.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/105.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/106.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/107.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/108.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/109.pdf
C:/Users/kevjy/Do

C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/196.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/197.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/198.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/199.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/2.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/20.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/200.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/201.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/202.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/203.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/204.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/205.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/206.pdf
C:/Users/kevjy/

C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/293.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/294.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/295.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/296.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/297.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/298.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/299.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/3.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/30.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/300.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/301.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/302.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/303.pdf
C:/Users/kevjy/

C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/390.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/391.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/392.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/393.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/394.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/395.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/396.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/397.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/398.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/399.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/4.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/40.pdf
C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/400.pdf
C:/Users/kevjy/

C:/Users/kevjy/Documents/Fall2017/EE379K/Labs/DSLab/Lab3/scrapedPDF/99.pdf


TypeError: 'str' object is not callable

![](http://i.markdownnotes.com/image_TWj4ap5.png)

#### Trying out a linear model:
Following a guide from https://www.kaggle.com/apapiu/regularized-linear-models

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

C:\Users\Kevin\Anaconda\lib\site-packages\matplotlib\__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
